# Neural Dual Solver 

This tutorial shows how to use `OTT` to compute the Wasserstein-4 optimal transport map between continuous measures in Euclidean space that are accessible via sampling.
{class}`~ott.solvers.nn.neuraldual.W2NeuralDual` solves this
problem by optimizing parameterized Kantorovich dual potential functions
and returning a  {class}`~ott.problems.linear.potentials.DualPotentials`
object that can be used to transport unseen source data samples to its target distribution (or vice-versa) or compute the corresponding distance between new source and target distribution.

The dual potentials can be specified as non-convex neural networks
({class}`~ott.solvers.nn.models.MLP`) or an
input-convex neural network ({class}`~ott.solvers.nn.models.ICNN`) {cite}`amos:17`.
{class}`~ott.solvers.nn.neuraldual.W2NeuralDual` implements the
method developed by {cite}`makkuva:20`
along with the improvements and fine-tuning of the
conjugate computation from {cite}`amos:23`.
For more insights on the approach itself, we refer the user
to the original sources.

In [ ]:
import sys

if "google.colab" in sys.modules:
    !pip install -q git+https://github.com/ott-jax/ott@main

In [ ]:
from dataclasses import dataclass
from functools import partial

from IPython.display import clear_output, display

import jax
import jax.numpy as jnp
import numpy as np
import optax
from torch.utils.data import DataLoader, IterableDataset

import matplotlib.pyplot as plt

from ott.geometry import pointcloud
from ott.problems.linear import potentials
from ott.problems.nn import dataset
from ott.solvers.nn import models, neuraldual
from ott.tools import plot, sinkhorn_divergence

## Setup Training and Validation Datasets

We apply the {class}`~ott.solvers.nn.neuraldual.W2NeuralDual` to compute the transport between toy datasets. In this tutorial, the user can choose between the datasets `simple` (data clustered in one center), `circle` (two-dimensional Gaussians arranged on a circle), `square_five` (two-dimensional Gaussians on a square with one Gaussian in the center), and `square_four` (two-dimensional Gaussians in the corners of a rectangle).

TODO

## Solve Neural Dual

In order to solve the neural dual, we need to define our dataloaders. The only requirement is that the corresponding source and target train and validation datasets are *iterators* that provide samples of batches from the source and target measures.

In [ ]:
dataloaders, input_dim = dataset.create(
    name_source="gaussian_simple", name_target="gaussian_circle"
)

In [ ]:
def plot_samples(eval_data_source, eval_data_target):
    fig, axs = plt.subplots(
        1, 2, figsize=(8, 4), gridspec_kw={"wspace": 0, "hspace": 0}
    )
    axs[0].scatter(
        eval_data_source[:, 0],
        eval_data_source[:, 1],
        color="#A7BED3",
        s=10,
        alpha=0.5,
    )
    axs[0].set_title("Source measure sample")
    axs[1].scatter(
        eval_data_target[:, 0],
        eval_data_target[:, 1],
        color="#1A254B",
        s=10,
        alpha=0.5,
    )
    axs[1].set_title("Target measure samples")

    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim(-6, 6)
        ax.set_ylim(-6, 6)
    return fig, ax


# Sample a batch for evaluation and plot it
eval_data_source = next(dataloaders.validloader_source)
eval_data_target = next(dataloaders.validloader_target)

plot_samples(eval_data_source, eval_data_target);

Next, we define the architectures parameterizing the dual potentials $f$ and $g$. We first parameterize $f$ with an {class}`~ott.solvers.nn.models.ICNN` and $\nabla g$ as a non-convex {class}`~ott.solvers.nn.models.MLP`. You can adapt the size of the ICNNs by passing a sequence containing hidden layer sizes. While ICNNs are by default containing partially positive weights, we can run the {class}`~ott.solvers.nn.neuraldual.W2NeuralDual` using approximations to this positivity constraint (via weight clipping and a weight penalization). For this, set `positive_weights` to `True` in both the ICNN architecture and {class}`~ott.solvers.nn.neuraldual.W2NeuralDual` configuration. For more details on how to customize the {class}`~ott.solvers.nn.models.ICNN` architectures, we refer you to the documentation.

In [ ]:
# initialize models and optimizers
num_train_iters = 5001

neural_f = models.ICNN(dim_data=2, dim_hidden=[128, 128])
neural_g = models.MLP(
    dim_hidden=[128, 128],
    is_potential=False,  # returns the gradient of the potential.
)

lr_schedule = optax.cosine_decay_schedule(
    init_value=5e-4, decay_steps=num_train_iters, alpha=1e-2
)
optimizer_f = optax.adam(learning_rate=lr_schedule, b1=0.5, b2=0.5)
optimizer_g = optax.adam(learning_rate=lr_schedule, b1=0.9, b2=0.999)

We then initialize the {class}`~ott.solvers.nn.neuraldual.W2NeuralDual` by passing the two {class}`~ott.solvers.nn.models.ICNN` models parameterizing $f$ and $g$, as well as by specifying the input dimensions of the data and the number of training iterations to execute. Once the {class}`~ott.solvers.nn.neuraldual.W2NeuralDual` is initialized, we can obtain the neural {class}`~ott.problems.linear.potentials.DualPotentials` by passing the corresponding dataloaders to it. As here our training and validation datasets do not differ, we pass (`dataloader_source`, `dataloader_target`) for both training and validation steps. For more details on how to configure the {class}`~ott.solvers.nn.neuraldual.W2NeuralDual`, we refer you to the documentation.

<font color='#F2545B'>Execution of the following cell will probably take a few minutes, depending on your system and the number of training iterations.</font>

In [ ]:
def training_callback(step, learned_potentials):
    # Callback function as the training progresses to visualize the couplings.
    if step % 1000 == 0:
        clear_output()
        print(f"Training iteration: {step}/{num_train_iters}")

        fig, ax = plot.plot_ot_map(
            learned_potentials, eval_data_source, eval_data_target, forward=True
        )
        display(fig)
        plt.close(fig)

        fig, ax = plot.plot_ot_map(
            learned_potentials,
            eval_data_source,
            eval_data_target,
            forward=False,
        )
        display(fig)
        plt.close(fig)

        fig, ax = plot.plot_potential(learned_potentials)
        display(fig)
        plt.close(fig)


neural_dual_solver = neuraldual.W2NeuralDual(
    input_dim,
    neural_f,
    neural_g,
    optimizer_f,
    optimizer_g,
    num_train_iters=num_train_iters,
)
learned_potentials = neural_dual_solver(
    *dataloaders,
    callback=training_callback,
)
clear_output()

The output of the solver, `learned_potentials`, is an instance of
{class}`~ott.problems.linear.potentials.DualPotentials`.
This  gives us access to the learned potentials and provides functions to compute and plot the forward and
inverse OT maps between the measures.

In [ ]:
plot.plot_potential(learned_potentials, forward=True)
plot.plot_potential(learned_potentials, forward=False);

## Evaluate Neural Dual

After training has completed successfully, we can evaluate the neural {class}`~ott.problems.linear.potentials.DualPotentials` on unseen incoming data. We first sample a new batch from the source and target distribution.

Now, we can plot the corresponding transport from source to target using the gradient of the learning potential $g$, i.e., $\nabla g(\text{source})$, or from target to source via the gradient of the learning potential $f$, i.e., $\nabla f(\text{target})$.

In [ ]:
plot.plot_ot_map(
    learned_potentials, eval_data_source, eval_data_target, forward=True
);

In [ ]:
plot.plot_ot_map(
    learned_potentials, eval_data_source, eval_data_target, forward=False
);

We further test, how close the predicted samples are to the sampled data.

First for potential $g$, transporting source to target samples. Ideally the resulting {class}`~ott.solvers.linear.sinkhorn.Sinkhorn` distance is close to $0$.

In [ ]:
@jax.jit
def sinkhorn_loss(x, y, epsilon=0.1):
    """Computes transport between (x, a) and (y, b) via Sinkhorn algorithm."""
    a = jnp.ones(len(x)) / len(x)
    b = jnp.ones(len(y)) / len(y)

    sdiv = sinkhorn_divergence.sinkhorn_divergence(
        pointcloud.PointCloud, x, y, epsilon=epsilon, a=a, b=b
    )
    return sdiv.divergence

In [ ]:
pred_target = learned_potentials.transport(eval_data_source)
print(
    f"Sinkhorn distance between source predictions and data samples: {sinkhorn_loss(pred_target, eval_data_target):.2f}"
)

Then for potential $f$, transporting target to source samples. Again, the resulting {class}`~ott.solvers.linear.sinkhorn.Sinkhorn` distance needs to be close to $0$.

In [ ]:
pred_source = learned_potentials.transport(eval_data_target, forward=False)
print(
    f"Sinkhorn distance between source predictions and data samples: {sinkhorn_loss(pred_source, eval_data_source):.2f}"
)

Besides computing the transport and mapping source to target samples or vice versa, we can also compute the overall distance between new source and target samples.

In [ ]:
neural_dual_dist = learned_potentials.distance(
    eval_data_source, eval_data_target
)
print(
    f"Neural dual distance between source and target data: {neural_dual_dist:.2f}"
)

Which compares to the primal {class}`~ott.solvers.linear.sinkhorn.Sinkhorn` distance in the following.

In [ ]:
sinkhorn_dist = sinkhorn_loss(eval_data_source, eval_data_target)
print(f"Sinkhorn distance between source and target data: {sinkhorn_dist:.2f}")

## Solving a harder problem

We next set up a harder OT problem to transport from a mixture
of five Gaussians to a mixture of four Gaussians and solve
it by using the non-convex {class}`~ott.solvers.nn.models.MLP`
potentials to model $f$ and $g$.

In [ ]:
dataloaders, input_dim = dataset.create(
    name_source="gaussian_square_five", name_target="gaussian_square_four"
)

eval_data_source = next(dataloaders.validloader_source)
eval_data_target = next(dataloader.validloader_target)
plot_samples(eval_data_source, eval_data_target);

In [ ]:
num_train_iters = 20001

neural_f = models.MLP(dim_hidden=[128, 128])
neural_g = models.MLP(dim_hidden=[128, 128])

lr_schedule = optax.cosine_decay_schedule(
    init_value=5e-4, decay_steps=num_train_iters, alpha=1e-2
)
optimizer_f = optax.adamw(learning_rate=lr_schedule)
optimizer_g = optimizer_f

neural_dual_solver = neuraldual.W2NeuralDual(
    input_dim,
    neural_f,
    neural_g,
    optimizer_f,
    optimizer_g,
    num_train_iters=num_train_iters,
)
learned_potentials = neural_dual_solver(
    *dataloaders,
    callback=training_callback,
)
clear_output()

We can run the same visualizations and Wasserstein-2 distance estimations as before:

In [ ]:
plot.plot_ot_map(
    learned_potentials, eval_data_source, eval_data_target, forward=True
)
plot.plot_ot_map(
    learned_potentials, eval_data_source, eval_data_target, forward=False
)

pred_target = learned_potentials.transport(eval_data_source)
print(
    f"Sinkhorn distance between target predictions and data samples: {sinkhorn_loss(pred_target, eval_data_target):.2f}"
)


pred_source = learned_potentials.transport(eval_data_target, forward=False)
print(
    f"Sinkhorn distance between source predictions and data samples: {sinkhorn_loss(pred_source, eval_data_source):.2f}"
)

neural_dual_dist = learned_potentials.distance(
    eval_data_source, eval_data_target
)
print(
    f"Neural dual distance between source and target data: {neural_dual_dist:.2f}"
)

sinkhorn_dist = sinkhorn_loss(eval_data_source, eval_data_target)
print(f"Sinkhorn distance between source and target data: {sinkhorn_dist:.2f}")